# Construccion del Dataframe

In [1]:
# Importamos los modulos y librerias que vamos a usar
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np

In [2]:
# Informamos identidad al servidor
head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
           'Chrome/59.0.3071.115 Safari/537.36'}

In [3]:
novelas = 'https://listas.20minutos.es/lista/novelas-ambientadas-en-ciudades-reales-334807/'

In [6]:
novelas_pag = rq.get(novelas, headers = head) 

In [7]:
novelas_pag

<Response [200]>

In [10]:
# Creamos el parser
soup = BeautifulSoup(rq.get(novelas).text, 'html.parser')

In [98]:
# Extraemos los titulos. Aca esta asociada la ciudad
titulo = re.findall('<h3>(.*?)</h3>', rq.get(novelas).text)

In [100]:
# Son 74 titulos. Esta OK
len(titulo)

74

In [196]:
# Extraemos la sinopsis
sinopsis = re.findall('<b>Sinopsis:</b>(.*?)<', rq.get(novelas).text)

In [197]:
# Faltan diez reseñas
len(sinopsis)

64

In [115]:
# Extraemos los puntajes de cada libro
puntaje = re.findall('<span class="recibidos">Ha recibido <strong>(.+?) puntos</strong>', rq.get(novelas).text)

In [117]:
# Son 74 puntuaciones. Esta OK
len(puntaje)

74

Tratamos de corregir la extraccion para conseguir las reseñas que faltan

In [193]:
# Extraemos el div con todo el contenido que nos interesa junto: Autor, Editorial y Reseña larga

descripcion_larga = soup.findAll('div', attrs={'class':'descripcion_elemento_larga'})

In [195]:
# Son 74 novelas. Esta OK
len(descripcion_larga)

73

Pasamos 'descripcion_larga' a string para pasarle una regex

In [184]:
des = str(lista)

In [216]:
# Buscamos de nuevo la sinopsis
sinox = re.findall('<b>Sinopsis:</b>(.*?)</div>', des)

In [218]:
# Sigue sin levantar todas las reseñas
len(sinox)

51

Sacamos la etiqueta '/b' para que la regex extraiga todo

In [251]:
# Reemplazamos
des45 = des.replace('<b>Sinopsis</b>', '<b>Sinopsis:</b>') 

In [256]:
# Extraemos
sinox456 = re.findall('<b>Sinopsis:</b>(.*?)<', des45)

In [257]:
# El largo ahora esta bien
len(sinox456)

73

Continuo con la limpieza de sinox 456. Si bien toma todas las reseñas, algunas las agarra por la mitad por cómo está armada la regex (corta en el '<' del br)

In [263]:
des458 = des45.replace('<br/>', ' ') 

In [265]:
des459 = des458.replace('\n', ' ')

In [266]:
des460 = des458.replace('\r', ' ')

In [271]:
des461 = des460.replace('\n  \n','')

In [276]:
des462 = des461.replace('  \n', '')

In [277]:
sinox462 = re.findall('<b>Sinopsis:</b>(.*?)<', des462)

In [278]:
len(sinox462)

73

In [280]:
# Ya estan las reseñas bien armadas
sinox462[1]

' Siglo XIV. La ciudad de Barcelona se encuentra en su momento de mayor prosperidad; ha crecido hacia la Ribera, el humilde barrio de los pescadores, cuyos habitantes deciden construir, con el dinero de unos y el esfuerzo de otros, el mayor templo mariano jamás conocido: Santa María de la Mar. Una construcción que es paralela a la azarosa historia de Arnau, un siervo de la tierra que huye de los abusos de su señor feudal y se refugia en Barcelona, donde se convierte en ciudadano y, con ello, en hombre libre. El joven Arnau trabaja como palafrenero, estibador, soldado y cambista. Una vida extenuante, siempre al amparo de la catedral de la Mar, que le iba a llevar de la miseria del fugitivo a la nobleza y la riqueza. Pero con esta posición privilegiada también le llega la envidia de sus pares, que urden una sórdida conjura que pone su vida en manos de la Inquisición... La catedral del mar es una trama en la que se entrecruzan lealtad y venganza, traición y amor, guerra y peste, en un mun

In [298]:
df_nov = pd.DataFrame(titulo)

In [299]:
df_nov['Puntaje'] = puntaje

In [306]:
# Faltaba una sinopsis en la lista
len(sinox462)

73

In [305]:
len(titulo)

74

In [352]:
# La ubicamos y la insertamos en la lista original
novela_faltante = "La genetista Belgin Doruk vuela de regreso a su Estambul natal después de varios años en Nueva York. Durante las más de diez horas de trayecto, azarada por sus dudas, no ha dejado de pensar en el paso que está dando, en lo que significa regresar a Turquía, sobre todo, porque lo hace por amor. Su novio, el escultor Ayhan Al Adana, la espera ilusionado y asustado a la vez, mientras mata el tiempo y los nervios conversando con el camarero del bar del aeropuerto. Justo cuando el vuelo de Belgin ha aterrizado, se anuncia que un fallo informático ha paralizado el sistema y el aeropuerto se convierte en un caos. Pero hay más gente en el aeropuerto, otros personajes cuyas vidas se entrecruzarán a lo largo de la novela."

In [364]:
sinopsis_completa = sinox462[:]

In [367]:
sinopsis_completa.insert(5,novela_faltante)

In [368]:
len(sinopsis_completa)

74

In [373]:
# La agregamos al dataframe
df_nov['Sinopsis'] = sinopsis_completa

In [376]:
# Renombramos la columna del titulo. Esta tambien tiene el nombre de la ciudad donde concurre la novela
df_nov.rename(columns={0:'Titulo'},inplace=True)

In [377]:
# Este es el DataFrame final con las novelas
df_nov

,Titulo,Puntaje,Sinopsis
0,La sombra del viento - Barcelona,2660,Un amanecer de 1945 un muchacho es conducido ...
1,La catedral del mar - Barcelona,2546,Siglo XIV. La ciudad de Barcelona se encuentr...
2,La ciudad de los ojos grises - Bilbao,2459,"Tras varios años viviendo en París, Alfredo G..."
3,Los hombres que no amaban a las mujeres - Esto...,1600,"Henrik Vanger, un anciano dueño de un importa..."
4,El prisionero del cielo - Barcelona,1553,"Barcelona, 1957. Daniel Sempere y su amigo Fe..."
5,El juego del ángel - Barcelona,1471,La genetista Belgin Doruk vuela de regreso a s...
6,Marina - Barcelona,1421,En la Barcelona de 1980 Óscar Drai sueña desp...
7,Memorias de África - Nairobi,1351,"Recién casada, una joven danesa, aristócrata ..."
8,El Perfume - París y Grasse,1330,Quizá los olores evoquen el privilegio de la ...
9,La judía más hermosa - Sevilla,1268,"Susana de Susón, la más hermosa hembra de la ..."


In [378]:
# Exportamos el csv
df_nov.to_csv('novelas.csv', sep='\t', encoding='utf-8')

In [380]:
# Levantamos el pdf
#pd.read_csv('novelas.csv', encoding='utf-8', engine='python', sep='\t')

# ETAPA1: PREPROCESAMIENTO

In [381]:
# Importamos NLTK 
import nltk

# Modulos
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

# Stopwords 
nltk.download('stopwords')
nltk.download('wordnet')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Federico\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Federico\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [382]:
# Instanciamos las stopwords (vocablos a excluir)

stop = set(stopwords.words('spanish'))

# Instanciamos los signos (signos de puntuacion a excluir)  

signos = set(string.punctuation) 

# Instanciamos el Lemmatizador de palabras 
lemma = WordNetLemmatizer()

# Creamos una funcion para...
def df_clean(doc):
    
    # Pasar a lowercase las palabras y splitearlas. A su vez quita las stopwords
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    # Vuelve a joinear por espacio entre palabras y saca las stopwords
    punc_free = ''.join(ch for ch in stop_free if ch not in signos)
    # Devuelve un listado de vocablos spliteados y lematizados (raiz de la palabra)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [384]:
# Probamos la funcion: limpio bien las stopwords 
for i in sinopsis_completa[1:5]:
    print(df_clean(i))

siglo xiv ciudad barcelona encuentra momento mayor prosperidad crecido hacia ribera humilde barrio pescadores cuyos habitantes deciden construir dinero esfuerzo otros mayor templo mariano jamás conocido santa maría mar construcción paralela azarosa historia arnau siervo tierra huye abusos señor feudal refugia barcelona convierte ciudadano y ello hombre libre joven arnau trabaja palafrenero estibador soldado cambista vida extenuante siempre amparo catedral mar iba llevar miseria fugitivo nobleza riqueza posición privilegiada llega envidia pares urden sórdida conjura pone vida manos inquisición catedral mar trama entrecruzan lealtad venganza traición amor guerra peste mundo marcado intolerancia religiosa ambición material segregación social ello convierte obra solo novela absorbente sino fascinante ambiciosa recreación luce sombras época feudal
tras varios años viviendo parís alfredo gastiasoro regresa bilbao enteron izarbe muerto retorno pretende ser último homenaje mujer amó pronto con

In [385]:
# Splitea vocablos con una lista por comprension
palabras = [df_clean(comment).split() for comment in sinopsis_completa] 

In [386]:
palabras

[['amanecer',
  '1945',
  'muchacho',
  'conducido',
  'padre',
  'misterioso',
  'lugar',
  'oculto',
  'corazón',
  'ciudad',
  'vieja',
  'cementerio',
  'libros',
  'olvidados',
  'allí',
  'daniel',
  'sempere',
  'encuentra',
  'libro',
  'maldito',
  'cambiará',
  'rumbo',
  'vida',
  'arrastrará',
  'laberinto',
  'intrigas',
  'secretos',
  'enterrados',
  'alma',
  'oscura',
  'ciudad',
  'sombra',
  'viento',
  'misterio',
  'literario',
  'ambientado',
  'barcelona',
  'primera',
  'mitad',
  'siglo',
  'xx',
  'últimos',
  'esplendores',
  'modernismo',
  'tinieblas',
  'posguerra'],
 ['siglo',
  'xiv',
  'ciudad',
  'barcelona',
  'encuentra',
  'momento',
  'mayor',
  'prosperidad',
  'crecido',
  'hacia',
  'ribera',
  'humilde',
  'barrio',
  'pescadores',
  'cuyos',
  'habitantes',
  'deciden',
  'construir',
  'dinero',
  'esfuerzo',
  'otros',
  'mayor',
  'templo',
  'mariano',
  'jamás',
  'conocido',
  'santa',
  'maría',
  'mar',
  'construcción',
  'paralela',


In [387]:
len(palabras)

74

# TOPIC MODELLING

In [388]:
import gensim
from gensim import corpora, models 
from gensim.corpora import Dictionary

C:\Users\Federico\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [389]:
# Le asignamos un indice a cada termino unico (tokens unicos). Se llama al metodo Dictionary de corpora
dct = corpora.Dictionary(palabras)

In [391]:
# Chequeamos el listado de tokens unicos. Son casi 180mil palabras unicas
print(dct)

Dictionary(3014 unique tokens: ['1945', 'allí', 'alma', 'amanecer', 'ambientado']...)


In [392]:
# Usando doc2bow de gensim convertimos el listado de token unicos en una matriz donde se cuenta la cantidad de veces que aparece cada termino en el cuerpo del texto analizado
matriz_terminounico_documento = [dct.doc2bow(comment) for comment in palabras]

In [393]:
# Evaluamos como queda la matriz: Hay muchos tokens unicos que aparecen solo una vez!
matriz_terminounico_documento[0:5]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1)],
 [(6, 2),
  (9, 1),
  (13, 1),
  (37, 1),
  (40, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 2),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 2),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 2),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),

In [394]:
# Instanciamos el modelo LDA
Lda = gensim.models.ldamodel.LdaModel

In [395]:
# Aplicamos el modelo pasandole como parametros la matriz de terminos, el numero de topicos, los terminos tokenizados y la cantidad de vueltas
ldamodel = Lda(matriz_terminounico_documento, num_topics=3, id2word = dct, passes=50)

In [396]:
# Vemos las 3 palabras mas probables por topico
print(ldamodel.print_topics(num_topics=5, num_words=3))

[(0, '0.007*"vida" + 0.005*"ciudad" + 0.004*"ser"'), (1, '0.006*"novela" + 0.005*"años" + 0.004*"amor"'), (2, '0.007*"vida" + 0.006*"años" + 0.005*"historia"')]


In [397]:
# Vemos la contribucion de cada topico a un documento. 
ldamodel[matriz_terminounico_documento[10]]

[(0, 0.98842567)]

In [398]:
# El topico definido por lda no parece nada loco: vida+ciudad+ser podria asociarse con iconos de la ciudad o la vida en ella
sinopsis_completa[10]

' Narra la historia de Daniel Rogers, un ex ludópata que se infiltra en una sociedad criminal que elige a sus víctimas mediante partidas clandestinas de blackjack haciendo que estos pierdan todo su dinero e incluso la vida, el bien más preciado que se apuestan sobre el tapete. Para seguir las pistas, Rogers tiene que recorrer la ciudad de Nueva York, donde se encontrará con los grandes iconos del cine y la literatura, como a Holly Golightly comiendo un croasán frente a Tiffany, la escena que interpreta Audrey Hepburn en Desayuno con diamantes. En esta geografía neoyorkina, Rogers también dará con algunos de los personajes que inventó Woody Allen, como Danny Rose, entre otros.'

In [399]:
# Estas son las palabras tokenizadas en base a las que se arma el lda
palabras[10]

['narra',
 'historia',
 'daniel',
 'rogers',
 'ex',
 'ludópata',
 'infiltra',
 'sociedad',
 'criminal',
 'elige',
 'víctimas',
 'mediante',
 'partidas',
 'clandestinas',
 'blackjack',
 'haciendo',
 'pierdan',
 'dinero',
 'incluso',
 'vida',
 'bien',
 'preciado',
 'apuestan',
 'tapete',
 'seguir',
 'pistas',
 'rogers',
 'recorrer',
 'ciudad',
 'nueva',
 'york',
 'encontrará',
 'grandes',
 'iconos',
 'cine',
 'literatura',
 'holly',
 'golightly',
 'comiendo',
 'croasán',
 'frente',
 'tiffany',
 'escena',
 'interpreta',
 'audrey',
 'hepburn',
 'desayuno',
 'diamante',
 'geografía',
 'neoyorkina',
 'rogers',
 'dará',
 'personajes',
 'inventó',
 'woody',
 'allen',
 'danny',
 'rose',
 'otros']

In [401]:
# Topico con mayor probabilidad: para este documento el topico mas probable es el 0
ldamodel.get_document_topics(bow=matriz_terminounico_documento[10], minimum_probability=0.4)

[(0, 0.98842549)]

In [402]:
# Creamos un dataframe con la probabilidad para cada topico por documento (o fragmento de libro)

probabilidad_por_topico = []
for bow in matriz_terminounico_documento:
    # Se indexa x[1] en la lista por comprension para agarrar el valor de la tupla en lugar del indice del topico
    probabilidad_por_topico.append(x[1] for x in ldamodel[bow])

topicos_fragmentos = pd.DataFrame(probabilidad_por_topico)

In [409]:
topicos_fragmentos.sample(20)

,0,1,2
72,0.990600,NaN,NaN
26,0.988167,NaN,NaN
15,0.989723,NaN,NaN
43,0.993792,NaN,NaN
63,0.989309,NaN,NaN
46,0.991588,NaN,NaN
61,0.992403,NaN,NaN
17,0.985385,NaN,NaN
34,0.984127,NaN,NaN
56,0.987965,NaN,NaN


In [410]:
# Creamos un dataframe completo, con los comentarios originales, y el topico mayoritario de pertenencia con idxmax de pandas
topicos = []

topicos = topicos_fragmentos.idxmax(axis=1)

In [411]:
# Creo el Dataframe con los topicos

df_topicos = pd.concat([df_nov, topicos], axis=1)

In [412]:
df_topicos

,Titulo,Puntaje,Sinopsis,0
0,La sombra del viento - Barcelona,2660,Un amanecer de 1945 un muchacho es conducido ...,0
1,La catedral del mar - Barcelona,2546,Siglo XIV. La ciudad de Barcelona se encuentr...,0
2,La ciudad de los ojos grises - Bilbao,2459,"Tras varios años viviendo en París, Alfredo G...",0
3,Los hombres que no amaban a las mujeres - Esto...,1600,"Henrik Vanger, un anciano dueño de un importa...",0
4,El prisionero del cielo - Barcelona,1553,"Barcelona, 1957. Daniel Sempere y su amigo Fe...",0
5,El juego del ángel - Barcelona,1471,La genetista Belgin Doruk vuela de regreso a s...,0
6,Marina - Barcelona,1421,En la Barcelona de 1980 Óscar Drai sueña desp...,0
7,Memorias de África - Nairobi,1351,"Recién casada, una joven danesa, aristócrata ...",0
8,El Perfume - París y Grasse,1330,Quizá los olores evoquen el privilegio de la ...,0
9,La judía más hermosa - Sevilla,1268,"Susana de Susón, la más hermosa hembra de la ...",0


In [413]:
df_topicos.rename(columns={0:'Topicos'},inplace=True)

In [414]:
df_topicos

,Titulo,Puntaje,Sinopsis,Topicos
0,La sombra del viento - Barcelona,2660,Un amanecer de 1945 un muchacho es conducido ...,0
1,La catedral del mar - Barcelona,2546,Siglo XIV. La ciudad de Barcelona se encuentr...,0
2,La ciudad de los ojos grises - Bilbao,2459,"Tras varios años viviendo en París, Alfredo G...",0
3,Los hombres que no amaban a las mujeres - Esto...,1600,"Henrik Vanger, un anciano dueño de un importa...",0
4,El prisionero del cielo - Barcelona,1553,"Barcelona, 1957. Daniel Sempere y su amigo Fe...",0
5,El juego del ángel - Barcelona,1471,La genetista Belgin Doruk vuela de regreso a s...,0
6,Marina - Barcelona,1421,En la Barcelona de 1980 Óscar Drai sueña desp...,0
7,Memorias de África - Nairobi,1351,"Recién casada, una joven danesa, aristócrata ...",0
8,El Perfume - París y Grasse,1330,Quizá los olores evoquen el privilegio de la ...,0
9,La judía más hermosa - Sevilla,1268,"Susana de Susón, la más hermosa hembra de la ...",0


# PREPROCESSING Y TOPIC MODELLING CON SKLEARN

In [415]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

In [418]:
stop_words = ["a","allí","libro","mayor","cuyos","jamás","y","iba","e","éste","autor","va","do","actualmente","acuerdo","adelante","ademas","además","adrede","afirmó","agregó","ahi","ahora","ahí","al","algo","alguna","algunas","alguno","algunos","algún","alli","allí","alrededor","ambos","ampleamos","antano","antaño","ante","anterior","antes","apenas","aproximadamente","aquel","aquella","aquellas","aquello","aquellos","aqui","aquél","aquélla","aquéllas","aquéllos","aquí","arriba","abajo","aseguró","asi","así","atras","aun","aunque","ayer","añadió","aún","b","bajo","bastante","bien","breve","buen","buena","buenas","bueno","buenos","c","cada","casi","cerca","cierta","ciertas","cierto","ciertos","cinco","claro","comentó","como","con","conmigo","conocer","conseguimos","conseguir","considera","consideró","consigo","consigue","consiguen","consigues","contigo","contra","cosas","creo","cual","cuales","cualquier","cuando","cuanta","cuantas","cuanto","cuantos","cuatro","cuenta","cuál","cuáles","cuándo","cuánta","cuántas","cuánto","cuántos","cómo","d","da","dado","dan","dar","de","debajo","debe","deben","debido","decir","dejó","del","delante","demasiado","demás","dentro","deprisa","desde","despacio","despues","después","detras","detrás","dia","dias","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","donde","dos","durante","día","días","dónde","e","ejemplo","el","ella","ellas","ello","ellos","embargo","empleais","emplean","emplear","empleas","empleo","en","encima","encuentra","enfrente","enseguida","entonces","entre","era","eramos","eran","eras","eres","es","esa","esas","ese","eso","esos","esta","estaba","estaban","estado","estados","estais","estamos","estan","estar","estará","estas","este","esto","estos","estoy","estuvo","está","están","ex","excepto","existe","existen","explicó","expresó","f","fin","final","fue","fuera","fueron","fui","fuimos","g","general","gran","grandes","gueno","h","ha","haber","habia","habla","hablan","habrá","había","habían","hace","haceis","hacemos","hacen","hacer","hacerlo","haces","hacia","haciendo","hago","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","horas","hoy","hubo","i","igual","incluso","indicó","informo","informó","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","j","junto","k","l","la","lado","largo","las","le","lejos","les","llegó","lleva","llevar","lo","los","luego","lugar","m","mal","manera","manifestó","mas","mayor","me","mediante","medio","mejor","mencionó","menos","menudo","mi","mia","mias","mientras","mio","mios","mis","misma","mismas","mismo","mismos","modo","momento","mucha","muchas","mucho","muchos","muy","más","mí","mía","mías","mío","míos","n","nada","nadie","ni","ninguna","ningunas","ninguno","ningunos","ningún","no","nos","nosotras","nosotros","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","os","otra","otras","otro","otros","p","pais","para","parece","parte","partir","pasada","pasado","paìs","peor","pero","pesar","poca","pocas","poco","pocos","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","podrá","podrán","podría","podrían","poner","por","porque","posible","primer","primera","primero","primeros","principalmente","pronto","propia","propias","propio","propios","proximo","próximo","próximos","pudo","pueda","puede","pueden","puedo","pues","q","qeu","que","quedó","queremos","quien","quienes","quiere","quiza","quizas","quizá","quizás","quién","quiénes","qué","r","raras","realizado","realizar","realizó","repente","respecto","s","sabe","sabeis","sabemos","saben","saber","sabes","salvo","se","sea","sean","segun","segunda","segundo","según","seis","ser","sera","será","serán","sería","señaló","si","sido","siempre","siendo","siete","sigue","siguiente","sin","sino","sobre","sois","sola","solamente","solas","solo","solos","somos","son","soy","soyos","su","supuesto","sus","suya","suyas","suyo","sé","sí","sólo","t","tal","tambien","también","tampoco","tan","tanto","tarde","te","temprano","tendrá","tendrán","teneis","tenemos","tener","tenga","tengo","tenido","tenía","tercera","ti","tiempo","tiene","tienen","toda","todas","todavia","todavía","todo","todos","total","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","trata","través","tres","tu","tus","tuvo","tuya","tuyas","tuyo","tuyos","tú","u","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","usted","ustedes","v","va","vais","valor","vamos","van","varias","varios","vaya","veces","ver","verdad","verdadera","verdadero","vez","vosotras","vosotros","voy","vuestra","vuestras","vuestro","vuestros","w","x","y","ya","yo","z","él","ésa","ésas","ése","ésos","ésta","éstas","éste","éstos","última","últimas","último","últimos"]


In [420]:
# El metodo CountVectorizer de sklearn hace todo el preprocesamiento junto, stopwords, lowercase;tokenizacion usando regex

vectorizer = CountVectorizer(min_df=3, max_df=0.9, 
                             stop_words=stop_words, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(sinopsis_completa)

In [421]:
lda_model2 = LatentDirichletAllocation(n_topics=3, max_iter=10, learning_method='online')

In [422]:
lda_2 = lda_model2.fit_transform(data_vectorized)

C:\Users\Federico\Anaconda2\envs\py36\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [424]:
print(lda_2[3],lda_2[3].sum())

[ 0.03276978  0.03862021  0.92861001] 1.0


In [425]:
print (vectorizer.vocabulary_)

{'padre': 194, 'coraz': 58, 'ciudad': 49, 'vieja': 276, 'libros': 155, 'olvidados': 191, 'all': 10, 'daniel': 64, 'cambiar': 36, 'rumbo': 235, 'vida': 274, 'intrigas': 145, 'secretos': 240, 'alma': 11, 'oscura': 193, 'misterio': 171, 'barcelona': 27, 'siglo': 245, 'posguerra': 211, 'barrio': 28, 'habitantes': 124, 'deciden': 67, 'dinero': 81, 'templo': 259, 'jam': 147, 'santa': 238, 'historia': 132, 'tierra': 265, 'convierte': 57, 'ciudadano': 50, 'hombre': 134, 'joven': 148, 'catedral': 43, 'tambi': 257, 'llega': 157, 'pone': 210, 'trama': 268, 'entrecruzan': 93, 'venganza': 272, 'traici': 267, 'amor': 15, 'guerra': 122, 'mundo': 178, 'social': 248, 'obra': 189, 'novela': 187, 'par': 196, 'regresa': 226, 'muerto': 173, 'ltimo': 160, 'mujer': 175, 'circunstancias': 48, 'muerte': 172, 'enfrentarse': 90, 'pasa': 197, 'convertirse': 56, 'ciudades': 51, 'viejo': 277, 'ambientada': 12, 'camino': 37, 'hist': 131, 'rico': 233, 'relato': 230, 'bella': 30, 'due': 85, 'importante': 140, 'grupo':

In [426]:
vectorizer.get_feature_names()

['abandonado',
 'acaba',
 'acci',
 'acompa',
 'actual',
 'aeropuerto',
 'afrontar',
 'agente',
 'alem',
 'alg',
 'all',
 'alma',
 'ambientada',
 'amigo',
 'amigos',
 'amor',
 'amores',
 'anuncia',
 'aparece',
 'aqu',
 'asesino',
 'atr',
 'avanza',
 'aventura',
 'ayuda',
 'bajos',
 'bar',
 'barcelona',
 'barrio',
 'barrios',
 'bella',
 'cad',
 'cadas',
 'caf',
 'calle',
 'calles',
 'cambiar',
 'camino',
 'capit',
 'capital',
 'casa',
 'caso',
 'casualidad',
 'catedral',
 'chica',
 'cielo',
 'cincuenta',
 'cine',
 'circunstancias',
 'ciudad',
 'ciudadano',
 'ciudades',
 'comienza',
 'condici',
 'conduce',
 'conoce',
 'convertirse',
 'convierte',
 'coraz',
 'culo',
 'cuya',
 'cuyas',
 'cuyo',
 'dando',
 'daniel',
 'deber',
 'decide',
 'deciden',
 'deja',
 'derecho',
 'desaparecido',
 'descubre',
 'desenga',
 'deseo',
 'despertar',
 'despu',
 'destino',
 'destinos',
 'detective',
 'dictadura',
 'diez',
 'dinero',
 'distintas',
 'don',
 'dudas',
 'due',
 'edad',
 'empieza',
 'encontrar',
 '

In [428]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model2, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      43.049466        1       1 -372.142456 -195.243271
1      39.470088        1       2 -437.769562 -443.703552
0      17.480446        1       3 -620.127258 -262.638367, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
274   Default  38.000000           vida  38.000000  30.0000  30.0000
33    Default   5.000000            caf   5.000000  29.0000  29.0000
152   Default   4.000000          kabul   4.000000  28.0000  28.0000
176   Default   5.000000        mujeres   5.000000  27.0000  27.0000
83    Default   5.000000            don   5.000000  26.0000  26.0000
163   Default   4.000000          madre   4.000000  25.0000  25.0000
40    Default   4.000000           casa   4.000000  24.0000  24.0000
108   Default   3.000000   experiencias   3.000000  23.0000  23.0000
236   Default   3.000000      sacerdote   3.000000  22.0000  22.0000
35    Default   5.000000         calles   5.000000  21.0000  21.0000
266   Default   5.000000            tom   5.000000  20.0000  20.0000
187   Default  26.000000         novela  26.000000  19.0000  19.0000
31    Default   4.000000            cad   4.000000  18.0000  18.0000
49    Default  27.000000         ciudad  27.000000  17.0000  17.0000
135   Default   3.000000        hombres   3.000000  16.0000  16.0000
143   Default   6.000000     inglaterra   6.000000  15.0000  15.0000
27    Default   7.000000      barcelona   7.000000  14.0000  14.0000
71    Default   4.000000       descubre   4.000000  13.0000  13.0000
15    Default  19.000000           amor  19.000000  12.0000  12.0000
129   Default   4.000000           hija   4.000000  11.0000  11.0000
223   Default   4.000000       realidad   4.000000  10.0000  10.0000
263   Default   7.000000           tico   7.000000   9.0000   9.0000
168   Default   5.000000         medida   5.000000   8.0000   8.0000
155   Default   6.000000         libros   6.000000   7.0000   7.0000
220   Default   5.000000          punto   5.000000   6.0000   6.0000
8     Default   3.000000           alem   3.000000   5.0000   5.0000
32    Default   4.000000          cadas   4.000000   4.0000   4.0000
217   Default   9.000000   protagonista   9.000000   3.0000   3.0000
268   Default   3.000000          trama   3.000000   2.0000   2.0000
204   Default   3.000000        persona   3.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
71     Topic3   1.974693       descubre   4.022077   1.0327  -4.8500
264    Topic3   1.640287        tiempos   3.342940   1.0321  -5.0355
6      Topic3   1.627158       afrontar   3.324352   1.0296  -5.0436
219    Topic3   1.645696         pueblo   3.384945   1.0229  -5.0322
141    Topic3   1.514960        indagar   3.351814   0.9500  -5.1150
129    Topic3   2.065899           hija   4.777196   0.9058  -4.8048
243    Topic3   1.432979          serie   3.360756   0.8917  -5.1706
266    Topic3   2.285758            tom   5.527023   0.8611  -4.7037
184    Topic3   1.390686         ngeles   3.363464   0.8609  -5.2006
34     Topic3   1.693597          calle   4.105534   0.8586  -5.0035
35     Topic3   2.296639         calles   5.572147   0.8578  -4.6989
237    Topic3   1.676713            san   4.121980   0.8446  -5.0136
218    Topic3   1.632467  protagonistas   4.052239   0.8349  -5.0403
53     Topic3   1.367106        condici   3.403551   0.8320  -5.2177
167    Topic3   1.640243     matrimonio   4.089154   0.8306  -5.0355
223    Topic3   1.926411       realidad   4.819443   0.8271  -4.8747
101    Topic3   1.623219      esperanza   4.089448   0.8201  -5.0460
274    Topic3   6.192570           vida  38.115225  -0.0732  -3.7070
194    Topic3   2.312357          padre   7.811063   0.5268  -4.6921
148    Topic3   3.578560          joven  17.586211   0.1519  -4.2554
10     Top